In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import BeamDynamics as bd
import copy
import ipywidgets as iwdg
import IPython.display as idsp

In [ ]:
plotFont = {
    'family' : 'sans-serif',
    'weight' : 'normal',
    'size'   : 12
}
matplotlib.rc('font', **plotFont)
plt.rc('legend', fontsize=10)
defaultColorCycle = plt.rcParams["axes.prop_cycle"]

In [ ]:
plotDefsBase = [
    {
        'varName1': 'x', 'varName2': 'y',
        'opacityHist': 0.5,
    },
    {
        'varName1': 't', 'varName2': 'x',
        'opacityHist': 0.5,
    },
    {
        'varName1': 't', 'varName2': 'y',
        'opacityHist': 0.5,
    },
    {
        'varName1': 'x', 'varName2': 'xp',
        'opacityHist': 0.5,
    },
    {
        'varName1': 'y', 'varName2': 'yp',
        'opacityHist': 0.5,
    },
    {
        'varName1': 'z', 'varName2': 'pz',
        'opacityHist': 0.5,
    },
    {
        'varName1': 't', 'varName2': 'Ekin',
        'opacityHist': 0.5,
    },
    {
        'varName1': 't', 'varName2': 'pz',
        'opacityHist': 0.5,
    },
]

In [ ]:
dummyTable = pd.DataFrame({'Parameter1': [1, 2], 'Parameter2': [3, 4], 'Parameter3': [5, 6]})
interfaceSymbols = {
    'Linac1In': chr(int('1F7E0', base=16)),
    'DampingRingIn': chr(int('1F7E2', base=16)),
    'Linac2In': chr(int('1F534', base=16)),
    'Linac2Out': chr(int('1F7E3', base=16)),
}
distrDatabase = {
    'Linac1In': {
        'GeneralParameters': dummyTable.T,
        'Positrons': {
            'CTSB-N02-F100-E06-S0.5-T5.0_FCTest_Pavel_SolC_CLICTW-OptionA08B7-Bc0.50': {
                'plotDefs': [
                    {
                        'lims1': (-25, 25.),   # [mm]
                        'lims2': (-25., 25.)   # [mm]
                    },
                    {
                        'lims1': (62.2, 62.5),   # [ns]
                    },
                    {},
                    {
                        'lims2': (-30., 30.),   # [mrad]
                    },
                    {
                        'lims2': (-30., 30.),   # [mrad]
                    },
                    {
                        'lims1': (9999., 10001.),   # [mm]
                        'lims2': (0., 400.)   # [MeV/c]
                    },
                    {},
                    {},
                ]
            },
            'CTSB-N02-F100-E06-S0.5-T5.0_HTSTest_JNov04_SolC_CLICTW-Ztc200-Ri15-Bc0.50': {
                'plotDefs': [
                    {
                        'lims1': (-25, 25.),   # [mm]
                        'lims2': (-25., 25.)   # [mm]
                    },
                    {
                        'lims1': (62.8, 63.1),   # [ns]
                    },
                    {},
                    {
                        'lims2': (-30., 30.),   # [mrad]
                    },
                    {
                        'lims2': (-30., 30.),   # [mrad]
                    },
                    {
                        'lims1': (9999., 10001.),   # [mm]
                        'lims2': (0., 400.)   # [MeV/c]
                    },
                    {},
                    {},
                ]
            },
            'CTSB-N02-F100-E06-S0.5-T5.0_HTSTest_JNov04_SolC_PSISW-Ztc200-Ri15-Bc1.50': {
                'plotDefs': [
                    {
                        'lims1': (-25, 25.),   # [mm]
                        'lims2': (-25., 25.)   # [mm]
                    },
                    {
                        'lims1': (58.6, 58.9),   # [ns]
                    },
                    {},
                    {
                        'lims2': (-30., 30.),   # [mrad]
                    },
                    {
                        'lims2': (-30., 30.),   # [mrad]
                    },
                    {
                        'lims1': (9999., 10001.),   # [mm]
                        'lims2': (0., 400.)   # [MeV/c]
                    },
                    {},
                    {},
                ]
            }
        },
        'Electrons': {
        },
    },
    'DampingRingIn': {
        'GeneralParameters': dummyTable.T,
    },
    'Linac2In': {
        'GeneralParameters': dummyTable.T,
    },
    'Linac2Out': {
        'GeneralParameters': dummyTable.T,
    },
}

# TODO: Decide where to put this and integrate plotDefsBase
for interface in distrDatabase.values():
    for particleType in ({k: v for k, v in interface.items() if k != 'GeneralParameters'}).values():
        for distr in particleType.values():
            distr['plotDefs'][1]['lims2'] = distr['plotDefs'][0]['lims1']   # [mm]
            distr['plotDefs'][2]['lims1'] = distr['plotDefs'][1]['lims1']   # [ns]
            distr['plotDefs'][2]['lims2'] = distr['plotDefs'][0]['lims2']   # [mm]
            distr['plotDefs'][3]['lims1'] = distr['plotDefs'][0]['lims1']   # [mm]
            distr['plotDefs'][4]['lims1'] = distr['plotDefs'][0]['lims2']   # [mm]
            distr['plotDefs'][6]['lims1'] = distr['plotDefs'][1]['lims1']   # [ns]
            distr['plotDefs'][6]['lims2'] = distr['plotDefs'][5]['lims2']   # [MeV]
            distr['plotDefs'][7]['lims1'] = distr['plotDefs'][6]['lims1']   # [ns]
            distr['plotDefs'][7]['lims2'] = distr['plotDefs'][5]['lims2']   # [MeV/c]

In [ ]:
def load_distr(distrName):
    sdfFilePath = './BeamDistrs/Positrons_200MeV_Yongke/' + distrName + '.dat.sdf_txt'
    distr = bd.load_standard_fwf(sdfFilePath)
    return distr

# FCC-ee Injector Beams App

In [ ]:
with open("FCCeeInjectorLayout.png", 'rb') as img:
    accelLayout_image = iwdg.Image(
        value=img.read(),
        format='png',
        width=1024,
    )
interfaceKeys = list(distrDatabase.keys())
interfaceOptions = [
    (interfaceSymbols[key] + ' ' + key, key) for key in interfaceKeys
]
interface_dropdown = iwdg.Dropdown(
    options=interfaceOptions,
    value=interfaceKeys[-1],
    description='Interface:',
    disabled=False,
    layout={'width': 'initial'},
)
particleType_dropdown = iwdg.Dropdown(
    description='Particle type:',
    disabled=False,
    layout={'width': 'initial'},
)
distr_dropdown = iwdg.Dropdown(
    description='Variant:',
    disabled=False,
    layout={'width': 'initial'},
)

generalParameters_out = iwdg.Output(overflow='auto')
generalParameters_out.layout.width = '1024px'
generalParameters_out.layout.margin='0 0 0 200px'
plot_out = []
for pDef in plotDefsBase:
    plot_out.append(iwdg.Output())
descr_out = iwdg.Output(overflow='auto')
descr_out.layout.width = '1024px'

def display_general_parameters(interfaceName):
    with generalParameters_out:
        if interfaceName == 'Linac1In':
            parametersDf = pd.read_excel(
                './BeamDistrs/FCCeeInjectorParametersCollection.xlsx',
                sheet_name=0, index_col=0
            )
            idsp.display(parametersDf)
            #idsp.display(distrDatabase[interfaceName]['GeneralParameters'])

def clear_out():
    for p_out in plot_out:
        p_out.clear_output()
    descr_out.clear_output()
    
def plot_all(distr, distrName, plotDefsDistr):
    #clear_out()
    plotDefs = [{**pDefBase, **pDefDistr} for pDefBase, pDefDistr in zip(plotDefsBase, plotDefsDistr)]
    for pDef, p_out in zip(plotDefs, plot_out):
        with p_out:
            bd.plot_distr([distr], [pDef], figHeight=7, figWidth=12.44)
            plt.show()
    with descr_out:
        idsp.display(distr.describe())
        
def interface_dropdown_eventhandler(change):
    interfaceSel = change.new
    generalParameters_out.clear_output()
    clear_out()
    display_general_parameters(interfaceSel)
    particleTypes = [k for k in distrDatabase[interfaceSel].keys() if k != 'GeneralParameters']
    particleType_dropdown.options = particleTypes
        
def particle_type_dropdown_eventhandler(change):
    interfaceSel = interface_dropdown.value
    partTypeSel = change.new
    clear_out()
    try:
        variantKeys = list(distrDatabase[interfaceSel][partTypeSel].keys())
    except KeyError:
        variantKeys = []
    distr_dropdown.options = variantKeys
        
def distr_dropdown_eventhandler(change):
    interfaceSel = interface_dropdown.value
    partTypeSel = particleType_dropdown.value
    distrSel = change.new
    clear_out()
    if distrSel is not None:
        distr = load_distr(distrSel)
        plotDefs = distrDatabase[interfaceSel][partTypeSel][distrSel]['plotDefs']
        filterSpecs = {
            'x': plotDefs[3]['lims1'],
            'xp': plotDefs[3]['lims2'],
            'y': plotDefs[4]['lims1'],
            'yp': plotDefs[4]['lims2'],
            't': plotDefs[7]['lims1'],
            'pz': plotDefs[7]['lims2'],
        }
        distr = bd.filter_distr(distr, filterSpecs)
        plot_all(distr, distrSel, plotDefs)

interface_dropdown.observe(interface_dropdown_eventhandler, names='value')
particleType_dropdown.observe(particle_type_dropdown_eventhandler, names='value')
distr_dropdown.observe(distr_dropdown_eventhandler, names='value')

tab_layout = iwdg.Layout(margin='25px 0 0 0')
outputs_tab = iwdg.Tab(plot_out+[descr_out], layout=tab_layout)
for tabInd, pDef in enumerate(plotDefsBase):
    outputs_tab.set_title(tabInd, pDef['varName1']+'-'+pDef['varName2'])
outputs_tab.set_title(len(plotDefsBase), 'Description')
dashboard = iwdg.VBox([
    accelLayout_image,
    interface_dropdown,
    generalParameters_out,
    particleType_dropdown,
    distr_dropdown,
    outputs_tab
])
idsp.display(dashboard)
interface_dropdown.value = interfaceKeys[0]